# Trend Mining

## Timeline and Popularity

In this notebook you will be able to analyze timeline and popularity. 
- Configurations for this notebook can be found in **Timeline.yaml** file inside the **Config** folder
- Make sure you follow the setup instructions on **Readme.md** and have installed all the packages required for this task

In [ ]:
import os
import yaml
import warnings
import pandas as pd
import seaborn as sns
from yaspin import yaspin
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import wilcoxon
from yaml.loader import SafeLoader


warnings.filterwarnings('ignore')

### Load Config file

In [ ]:
with open('../Config/Miners.yaml') as f:
    config = yaml.load(f, Loader=SafeLoader)
print('General Config:', config)

In [ ]:
with open('../Config/Timeline.yaml') as f:
    dendogramConfig = yaml.load(f, Loader=SafeLoader)
print('Tineline Config:', dendogramConfig)

### Common function and class

In [ ]:
def readFile(file, path):
    try:
        spinner = yaspin()
        complete_path = f'{os.path.dirname(os.path.abspath(os.getcwd()))}\\{path}\\{file}'
        file_data = pd.read_csv(complete_path, index_col=0)
        spinner.write("✔️ File loaded.")
        spinner.stop()
        return file_data
    except Exception as e:
        print('Error reading file',e)

#### Common class

In [ ]:
class TimelineAndPopularity():
    """This is the class implementation for timeline and popularity
    """
    def __init__(self, data_frame):
        self.data_frame = data_frame 
        self.dirName = ""
        self.spinner = yaspin()

    def createOutputDir(self, dirName):
        """This function creates the folder to store the output graphs and images

        Args:
            dirName (str): Name of the output folder
        """
        self.dirName = dirName
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        does_folder_exist = os.path.exists(complete_path)
        if (does_folder_exist):
            self.spinner.write("✔️ Output directory already exists.")
        else:
            os.makedirs(complete_path)
            self.spinner.write("✔️ Folder created for output storage")

    def popularityByYears(self):
        """This function calculated the popularity over the years
        """
        self.data_frame['Date'] = self.data_frame['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d', yearfirst=True)).dt.date
        years = pd.DatetimeIndex(self.data_frame['Date']).year
        year_count =  Counter(years)
        print('Yearly distributions:', year_count)
        sns.lineplot(data=year_count).set(title=f"{self.dirName} yearly popularity")
        plt.xticks(rotation=90)
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_yearly_popularity.png"))
        self.spinner.write(f'✔️ Figure saved')

    def dailyTrend(self):
        """This function calculates the daily trend
        """
        days =  pd.to_datetime(self.data_frame['Date']).dt.date 
        days_count =  Counter(days) 
        sns.lineplot(data=days_count).set(title=f"{self.dirName} daily trend")
        plt.xticks(rotation=90)
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_daily_trend.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')

    def citationAnalysis(self):
        """This function calculates boxplot for the citations 
        """
        citations = self.data_frame['Cites']
        sns.boxplot(citations, orient='h').set(title=f"{self.dirName} citation boxPlot")
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_citation_boxPlot.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')
    
    def citationSummary(self):
        """This function calculates statistical summary for citations
        """
        citations = self.data_frame['Cites']
        print('Citation Summary:')
        print(citations.describe())

    def citationViolinPlot(self):
        """This function calculates the violin plot for citations
        """
        citations = self.data_frame['Cites']
        sns.violinplot(x=citations).set(title=f"{self.dirName} citation violinPlot")
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_citation_violinPlot.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')
    
    def plotOldvsNewCitations(self):
        """This function calculated old and new citations based on median of date
        """
        mid_date = self.data_frame['Date'].astype('datetime64[ns]').quantile(0.5, interpolation="midpoint")
        new_data = self.data_frame[self.data_frame['Date']>= mid_date]
        old_data = self.data_frame[self.data_frame['Date'] < mid_date]
        print('Median date is:',mid_date)    
       
        # Box plots
        fig, axes = plt.subplots(1, 2)
        sns.boxplot(old_data['Cites'],  ax=axes[0])
        axes[0].set_title("Old Data")
        sns.boxplot(new_data['Cites'],  ax=axes[1])
        axes[1].set_title("New Data")
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_oldVSnew_boxPlot.png"))
        self.spinner.write(f'✔️ Figure saved')

        # Voilen plots
        fig, axes = plt.subplots(1, 2)
        sns.violinplot(old_data['Cites'],  ax=axes[0])
        axes[0].set_title("Old Data")
        sns.violinplot(new_data['Cites'],  ax=axes[1])
        axes[1].set_title("New Data")
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_oldVSnew_violinPlot.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')


        # Summary of old and new
        print("Old Data Summary")
        print(old_data['Cites'].describe())
        print("New Data Summary")
        print(new_data['Cites'].describe())

    def titleLengthAnalysis(self):
        """This function calculates old and new titles based on median of the length of title
        """
        title_lens = self.data_frame['Title'].str.len()
        median_len = title_lens.median()
        longer_length_data = self.data_frame[self.data_frame['Title'].str.len() >= median_len]
        shorter_length_data = self.data_frame[self.data_frame['Title'].str.len() < median_len]
        print('Total data points', len(self.data_frame))
        print('Total data points with longer title', len(longer_length_data))
        print('Total data points with shorter title', len(shorter_length_data))

        # Box plots
        fig, axes = plt.subplots(1, 2)
        sns.boxplot(longer_length_data['Cites'],  ax=axes[0])
        axes[0].set_title("Longer Data")
        sns.boxplot(shorter_length_data['Cites'],  ax=axes[1])
        axes[1].set_title("Shorter Data")
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_titleLength_boxPlot.png"))
        self.spinner.write(f'✔️ Figure saved')
        

        # Voilen plots
        fig, axes = plt.subplots(1, 2)
        sns.violinplot(longer_length_data['Cites'],  ax=axes[0])
        axes[0].set_title("Longer Data")
        sns.violinplot(shorter_length_data['Cites'],  ax=axes[1])
        axes[1].set_title("Shorter Data")
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_titleLength_violinPlot.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')

        # Summary of old and new
        print("Longer Data Summary")
        print(longer_length_data['Cites'].describe())
        print("Shorter Data Summary")
        print(shorter_length_data['Cites'].describe())

        # Wilcoxon
        w, p = wilcoxon(longer_length_data['Cites'])
        print('Wilcoxon for longer')
        print('W:', w, 'P:', p)
        w, p = wilcoxon(shorter_length_data['Cites'])
        print('Wilcoxon for shorter')
        print('W:', w, 'P:', p)

    def fourWaySplit(self):
        """This function splits the title in four parts based on the four quantiles.
        """
        title_lens = self.data_frame['Title'].str.len()
        q1 = self.data_frame[self.data_frame['Title'].str.len() == title_lens.quantile(0.25, interpolation='midpoint')]
        q2 = self.data_frame[self.data_frame['Title'].str.len() > title_lens.quantile(0.25, interpolation='midpoint')]
        q3 = self.data_frame[self.data_frame['Title'].str.len() <= title_lens.quantile(0.50,  interpolation='midpoint')]
        q4 = self.data_frame[self.data_frame['Title'].str.len() > title_lens.quantile(0.75,  interpolation='midpoint')]
        print('Total length of data:', len(self.data_frame))
        print('Length of q1:', len(q1))
        print('Length of q2:', len(q2))
        print('Length of q3:', len(q3))
        print('Length of q4:', len(q4))
        
        # Box plots
        fig, axes = plt.subplots(1, 4)
        sns.boxplot(q1['Cites'],  ax=axes[0])
        axes[0].set_title("Q1 Data")
        sns.boxplot(q2['Cites'],  ax=axes[1])
        axes[1].set_title("Q2 Data")
        sns.boxplot(q3['Cites'],  ax=axes[2])
        axes[2].set_title("Q3 Data")
        sns.boxplot(q4['Cites'],  ax=axes[3])
        axes[3].set_title("Q4 Data")
        complete_path = f"{os.path.dirname(os.path.abspath(os.getcwd()))}\\{config['OUTPUT_PATH']}\\Timeline\\{self.dirName}"
        plt.savefig(os.path.join(complete_path, f"{self.dirName}_FourwaySplit_boxPlot.png"))
        plt.show()
        self.spinner.write(f'✔️ Figure saved')
        
        #Summary of data
        print('Q1 Cites Summary:')
        print(q1['Cites'].describe())
        print('Q2 Cites Summary:')
        print(q2['Cites'].describe())
        print('Q3 Cites Summary:')
        print(q3['Cites'].describe())
        print('Q4 Cites Summary:')
        print(q4['Cites'].describe())

    def getTopArticles(self):
        """This function calculates top article based on the citation
        """
        sorted_df = self.data_frame.sort_values(by=['Cites'],ascending=False) 
        combined = pd.DataFrame({'Title':sorted_df['Title_clean'], 'cites':sorted_df['Cites']})
        print('Top 5 articles')
        print(combined.head(5))
    

### Reddit

In [ ]:
reddit_data = readFile(config['REDDIT_DATA_CSV'], config['STORAGE_PATH'])

In [ ]:
reddit_timeline = TimelineAndPopularity(reddit_data)

In [ ]:
reddit_timeline.createOutputDir('Reddit')

In [ ]:
reddit_timeline.popularityByYears()

In [ ]:
reddit_timeline.dailyTrend()

In [ ]:
reddit_timeline.citationAnalysis()

In [ ]:
reddit_timeline.citationSummary()

In [ ]:
reddit_timeline.citationViolinPlot()

In [ ]:
reddit_timeline.plotOldvsNewCitations()

In [ ]:
reddit_timeline.titleLengthAnalysis()

In [ ]:
reddit_timeline.fourWaySplit()

In [ ]:
reddit_timeline.getTopArticles()

### Stackoverflow

In [ ]:
stackoverflow_data = readFile(config['STACKOVERFLOW_DATA_CSV'], config['STORAGE_PATH'])

In [ ]:
stackoverflow_timeline = TimelineAndPopularity(stackoverflow_data)

In [ ]:
stackoverflow_timeline.createOutputDir('Stackoverflow')

In [ ]:
stackoverflow_timeline.popularityByYears()

In [ ]:
stackoverflow_timeline.dailyTrend()

In [ ]:
stackoverflow_timeline.citationAnalysis()

In [ ]:
stackoverflow_timeline.citationSummary()

In [ ]:
stackoverflow_timeline.citationViolinPlot()

In [ ]:
stackoverflow_timeline.plotOldvsNewCitations()

In [ ]:
stackoverflow_timeline.titleLengthAnalysis()

In [ ]:
stackoverflow_timeline.fourWaySplit()

In [ ]:
stackoverflow_timeline.getTopArticles()

### Scopus

In [ ]:
scopus_data = readFile(config['SCOPUS_DATA_CSV'], config['STORAGE_PATH'])

In [ ]:
scopus_timeline = TimelineAndPopularity(scopus_data)

In [ ]:
scopus_timeline.createOutputDir('Scopus')

In [ ]:
scopus_timeline.popularityByYears()

In [ ]:
scopus_timeline.dailyTrend()

In [ ]:
scopus_timeline.citationAnalysis()

In [ ]:
scopus_timeline.citationSummary()

In [ ]:
scopus_timeline.citationViolinPlot()

In [ ]:
scopus_timeline.plotOldvsNewCitations()

In [ ]:
scopus_timeline.titleLengthAnalysis()

In [ ]:
scopus_timeline.fourWaySplit()

In [ ]:
scopus_timeline.getTopArticles()